# Bài giảng về học Deep Neural Network 
## MaSSP 2017, Computer Science
### Chuẩn bị: Nguyễn Vương Linh, MIT Class of 2017

Các bài giảng trước (Slide 3 & 4) đã đề cập đến Deep Neural Network (DNN) và ứng dụng trong một số bài toán Machine Learning cơ bản. Trong bài giảng này, chúng ta tập trung vào vấn đề _học_ DNN: làm sao để xác định được giá trị các trọng số có trong DNN. Chúng ta giả sử cấu trúc mạng DNN được biết trước: có bao nhiêu lớp trong DNN và mỗi lớp có bao nhiêu neurons. Slide 6 & 7 sẽ hướng dẫn các bạn cụ thể hơn làm sao để chọn được mạng DNN phù hợp trong từng bài toán cụ thể.

Một số kí hiệu được sử dụng trong bài giảng:
1. D: mạng DNN.
2. L: số lớp có trong D, đánh số từ 0. Cụ thể hơn, lớp thứ 0 tương ứng với dữ liệu và lớp thứ L tương ứng với nhãn.
3. $m_i$ ($i = 0, 1, .., L$): số neuron có trong lớp thứ $i$. Đôi khi sẽ sử dụng $m = (m_0, m_1, .., m_L)$ để kí hiệu số neuron có trong từng lớp.
4. $w^{(i)}_{jk}$: trọng số của cạnh nối neuron thứ $j$ trong lớp $i$ đến neuron thứ $k$ trong lớp $i + 1$, và $b^i_j$ là thiên lệch của neuron thứ $j$ trong lớp $i$.

__Bài tập__: sử dụng các kí hiệu mô tả như trên, hãy vẽ một DNN với $L = 3, m = (5, 3, 3, 2)$ và trọng số $w$ tuỳ ý.

# 0. Giải thích học DNN trong 10 dòng
Cho đến giờ, trong tay bạn là $n$ bộ dữ liệu $x_1, x_2, .., x_n$ và nhãn tương ứng là $y_1, y_2, .., y_n$. Làm sao để xác định được trọng số $w$? Có lẽ nhiều bạn sẽ hình dung được cách làm nói chung là như sau:

1. Thử từng bộ $(x_i, y_i)$.
2. Thuật toán feedforward sử dụng giá trị $x_i$ để tính kết quả $D(x_i)$ ở lớp cuối cùng.
3. So sánh $D(x_i)$ với $y_i$.
4. Sử dụng sai số để điều chỉnh lại trọng số $w^{(i)}_{jk}$ một cách hợp lý.
5. Lặp lại bước 1 đến khi bạn cảm thấy ưng ý!

Hiển nhiên triển khai ý tưởng nói trên một cách chi tiết đòi hỏi nhiều công sức. Slide 4 đã đề cập bước 2, do đó trong slide này mình sẽ đi sâu vào bước 3, 4 và 5, cũng như đề cập những vấn đề phát sinh khi học DNN. Trong các tài liệu chuyên sâu về DNN, thuật toán nói trên được biết đến với tên _truyền ngược sai số_ (backpropagation), với ý tưởng sử dụng sai số giữa $D(x_i)$ và $y_i$ để điều chỉnh lại các trọng số $w$ dùng để tính toán $D(x_i)$.

# 1. Giải thích chi tiết
## 1.1. Sử dụng sai số nào?
Như đã đề cập ở Slide 3 & 4, với dữ liệu $x_i$, $D(x_i)$ không trực tiếp trả lại kết quả (nhãn nào được chọn), mà trả ra một vector biểu diễn _phân phối xác suất p_, với $p_j$ là xác suất chọn nhãn thứ $j$, và nhãn của $D(x_i)$ là phần tử $k$ với giá trị $p_k$ lớn nhất. 

Tuy nhiên, không phải phân phối nào cũng giống nhau, kể cả trong số những phân phối cho ra cùng kết quả. Giả sử $(1, 0, 0)$ là vector biểu diễn nhãn thứ nhất. Rõ ràng bạn sẽ tin tưởng phân phối $(0.9, 0.05, 0.05)$ hơn là phân phối $(0.5, 0.25, 0.25)$. Kể cả nếu như phân phối cho ra kết quả sai, bạn vẫn tin tưởng phân phối $(0.3, 0.3, 0.4)$ hơn là phân phối $(0, 1, 0)$.

Trong toán học, bạn có thể mô tả khái niệm nói trên bằng _khoảng cách_ giữa hai phân phối, lấy ý tưởng dựa trên khoảng cách giữa hai điểm trên hệ trục toạ độ. Trong không gian Descartes, khoảng cách giữa 2 điểm $(r_1, s_1)$ và $(r_2, s_2)$ là $$\sqrt{(r_1 - r_2)^2 + (s_1 - s_2)^2}$$

Trong Slide này, chúng ta sẽ sử dụng _bình phương_ khoảng cách giữa phân phối $D(x_i)$ và $y_i$, kí hiệu bằng $\delta(D(x_i), y_i$): $$\delta(D(x_i), y_i) = ||D(x_i) - y_i||_2^2 = \sum_{j = 1}^m |D(x_i)_j - y_{ij}|^2$$

Có hai lí do để sử dụng bình phương khoảng cách, thay cho tổng khoảng cách $\sum_{j = 1}^m |D(x_i)_j - y_{ij}|$:
* Bạn muốn đánh lỗi nặng hơn những giá trị xa giá trị thực hơn.
* Bình phương khoảng cách giúp việc tính đạo hàm trở nên đơn giản.

Lưu ý rằng bình phương khoảng cách chỉ là một trong rất nhiều cách tính sai số. Trong Lab 3 & 4 cách tính sai số cross-entropy sẽ được sử dụng. 

## 1.2. Điều chỉnh trọng số
Với mỗi bộ $(x, y)$, chúng ta tính toán:

* Giá trị nhận được ở lớp cuối cùng $a^L$. Lưu ý là chúng ta khởi tạo $a^0 = x$, $z^i = w^{i - 1} a^{i - 1} + b^i$ và $a^i = \sigma(z^i)$ với $i = 0, 1, .., L$.

* Đặt sai số $C = \frac{1}{2n}||y - a^L(x)||$ và tính toán sai số ở lớp cuối cùng: $$\delta^L = \nabla_a C \odot \sigma'(z^L)$$.

* Truyền ngược sai số: tính ngược với $l = L - 1, L - 2, ..$, đặt $\delta^l = ((w^l)^T \delta^{l + 1}) \odot \sigma'(z^l)$.

* Giá trị gradient được tính theo công thức $$\frac{\partial C}{\partial w^l_{jk}} = a_k^l \delta^l_j$$ và $$\frac{\partial C}{\partial b^l_j} = \delta^l_j$$. 

* Điều chỉnh trọng số $w$ và thiên lệch $b$ tương ứng, ví dụ: $$b^i_j = b^i_j - \alpha \frac{\partial C}{\partial b^l_j}$$

Hằng số $\alpha > 0$ là _tốc độ học_, thể hiện sự thay đổi hệ số ứng với thông tin mới nhận được. Nếu chọn $\alpha$ quá nhỏ, hệ số trong mạng DNN sẽ không tiếp nhận thông tin mới. Nếu chọn $\alpha$ quá lớn, mạng DNN sẽ bị _overfit_ (điều chỉnh để khớp với dữ liệu cho trước). Một trong những cách tránh cả hai vấn đề này là thiết lập $\alpha$ lớn lúc ban đầu, và giảm dần $\alpha$ khi lượng dữ liệu tăng thêm.

Đoạn chương trình sau đây (trích từ http://neuralnetworksanddeeplearning.com/chap2.html) minh hoạ thuật toán truyền ngược (lưu ý chương trình này chỉ mang tính minh hoạ).

In [1]:
class Network(object):
    def backprop(self, x, y):
        """Return a tuple "(nabla_b, nabla_w)" representing the
        gradient for the cost function C_x.  "nabla_b" and
        "nabla_w" are layer-by-layer lists of numpy arrays, similar
        to "self.biases" and "self.weights"."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y) 

def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

# 2. Tóm tắt các phương trình trong truyền ngược

$\delta^L = \nabla_a C \odot \sigma'(z^L)$

$\delta^l = ((w^L)^T \delta^{l + 1}) \odot'(z^l)$

$\frac{\partial C}{\partial b^l_j} = \delta^l_j$

$\frac{\partial C}{\partial w^l_{jk}} = a^l_k \delta^l_j$